Download data for Susceptibility Mapping



In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [2]:
# inputs
startDate = '2022-01-01'
endDate = '2022-12-31'
aoi = ee.Geometry.Polygon(
        [[[100.21258493926561, 14.101686931777209],
          [100.21258493926561, 13.533597643942972],
          [100.92944284942186, 13.533597643942972],
          [100.92944284942186, 14.101686931777209]]])


Datasets:

1. FABDEM 30m: https://gee-community-catalog.org/projects/fabdem/
2. Soil grid 250m: https://gee-community-catalog.org/projects/isric/?h=soil
3. Landsat-9 2022 30m: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2
4. LULC 2022 10m: https://gee-community-catalog.org/projects/S2TSLULC/
5. Roads (south-east-asia) Vector: https://gee-community-catalog.org/projects/grip/?h=road
6. Shoreline Vector: https://gee-community-catalog.org/projects/shoreline/?h=cost
7. Rivers Vector: https://gee-community-catalog.org/projects/hydroatlas/?h=hydr
8. Rainfall Daily 5566m: https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY


In [3]:
# Applies scaling factors.
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    return image.addBands(opticalBands, None, True)\
              .addBands(thermalBands, None, True)

def remapper(image):
        remapped = image.remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9])
        return remapped

# datasets
dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM")\
        .filterBounds(aoi)\
        .mosaic()\
        .clip(aoi)

l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
        .filterBounds(aoi)\
        .filterDate(startDate, endDate)\
        .map(applyScaleFactors)\
        .filter(ee.Filter.lt('CLOUD_COVER', 10))\
        .median()\
        .clip(aoi)

ndvi = l9.normalizedDifference(['SR_B5', 'SR_B4'])
ndwi = l9.normalizedDifference(['SR_B3', 'SR_B5'])
ndbi = l9.normalizedDifference(['SR_B6', 'SR_B5'])
ndsi = l9.normalizedDifference(['SR_B2', 'SR_B5'])

lulc = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")\
        .filterBounds(aoi)\
        .filterDate(startDate, endDate)\
        .map(remapper)\
        .mosaic()\
        .clip(aoi)

soil = ee.Image("projects/sat-io/open-datasets/FAO/HWSD_V2_SMU")

roads = ee.FeatureCollection("projects/sat-io/open-datasets/GRIP4/South-East-Asia")\
        .filterBounds(aoi)

shoreline = ee.FeatureCollection('projects/sat-io/open-datasets/shoreline/mainlands')\
        .filterBounds(aoi)


rivers = ee.FeatureCollection("projects/sat-io/open-datasets/HydroAtlas/RiverAtlas_v10")\
        .filterBounds(aoi)


rainfall_ic = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
        .filterBounds(aoi)\
        .filterDate(startDate, endDate)


In [17]:
# checking the projection of the data

dem_proj = dem.projection()
l9_proj = l9.projection()
lulc_proj = lulc.projection()
soil_proj = soil.projection()
roads_proj = roads.first().geometry().projection()

print(dem_proj.getInfo())
print(l9_proj.getInfo())
print(lulc_proj.getInfo())
print(soil_proj.getInfo())
print(roads_proj.getInfo())

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.008084837557075694, 0, -180.00082337073326, 0, -0.008084837557075694, 90.00041168536663]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [20]:
roads.size().getInfo()

87849

In [21]:
Map = geemap.Map()

Map.addLayer(dem, {}, 'DEM')
Map.addLayer(ndbi, {'min':0, 'max':1, 'palette':['blue', 'yellow', 'red']}, 'NDBI')
Map.addLayer(ndvi, {'min':0, 'max':1, 'palette':['blue', 'yellow', 'green']}, 'NDVI')
Map.addLayer(roads, {}, 'Roads')
Map.addLayer(shoreline, {}, 'Shoreline')
Map.addLayer(rivers, {}, 'Rivers')
Map.centerObject(aoi, 12)
Map

Map(center=[13.817786844729987, 100.57101389434429], controls=(WidgetControl(options=['position', 'transparent…

In [22]:
# Description: This script generates the predictor variables for the model.

# Combine rivers and shoreline into a single FeatureCollection
rivers_and_shoreline = rivers.merge(shoreline)

# Generate distance rasters
def distance_to_feature(feature_collection, scale):
    # Convert the FeatureCollection to a FeatureCollection
    feature_collection = ee.FeatureCollection(feature_collection)

    # Use the distance function to generate a distance raster
    distance_raster = feature_collection.distance()

    # Reproject the raster to match the specified scale
    print(distance_raster.projection().getInfo())
    distance_raster = distance_raster.reproject(crs='EPSG:4326', scale=scale)

    # Clip the raster to the AOI
    distance_raster = distance_raster.clip(aoi)

    return distance_raster

# Generate distance rasters for roads, and rivers+shoreline
roads_distance = distance_to_feature(roads, 30)
#rivers_and_shoreline_distance = distance_to_feature(rivers_and_shoreline, 30)



{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [24]:
Map.addLayer(roads_distance, {'min':0, 'max':10000}, 'Roads distance (m)')
Map

Map(bottom=60767.0, center=[13.78608981978479, 100.58933074538237], controls=(WidgetControl(options=['position…

In [27]:

# Calculate Terrain variables
slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)
#curvature = ee.Terrain.zeroCrossing(dem)

# Calculate Topographic Wetness Index
#slope_rad = slope.multiply(3.141592653589793).divide(180)
#twi = ee.Terrain.wetness(dem, slope_rad)

Map.addLayer(slope, {'min':0, 'max':60}, 'slope')
Map.addLayer(aspect, {'min':0, 'max':360}, 'aspect')
#Map.addLayer(curvature, {'min':-1, 'max':1}, 'curvature')
#Map.addLayer(twi, {'min':0, 'max':20}, 'twi')
Map

Map(bottom=483972.453125, center=[13.808742584307161, 100.59768676757814], controls=(WidgetControl(options=['p…

In [ ]:

# Export to Google Drive
def export_to_drive(image, description, folder, scale):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=aoi.geometry().bounds().getInfo()['coordinates'],
        fileFormat='GeoTIFF'
    )
    task.start()

# Export all rasters to Google Drive
export_to_drive(roads_distance, 'roads_distance', 'your-folder-name', 30)
export_to_drive(rivers_and_shoreline_distance, 'rivers_and_shoreline_distance', 'your-folder-name', 30)
export_to_drive(slope, 'slope', 'your-folder-name', 30)
export_to_drive(aspect, 'aspect', 'your-folder-name', 30)
export_to_drive(curvature, 'curvature', 'your-folder-name', 30)
export_to_drive(twi, 'twi', 'your-folder-name', 30)
